### testing yelp api

In [1]:
!python --version

Python 3.6.3 :: Anaconda, Inc.


In [2]:
import os
import sys
import csv
import requests
import urllib.request
import json
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

sys.path.append('../')
from envir import config
import imp
imp.reload(config)

<module 'envir.config' from '../envir/config.py'>

In [3]:
if os.path.exists(config.shared+'yelp_categories.json') == False: 
    # download json of categories
    cat_url = 'https://www.yelp.com/developers/documentation/v3/all_category_list/categories.json'
    urllib.request.urlretrieve(cat_url, config.shared+'yelp_categories.json')

In [4]:
with open(config.shared+'yelp_categories.json') as f:
    data = json.load(f)

In [5]:
con = psycopg2.connect(config.con)
zipcodes = pd.read_sql("select * from stag.ny_zipcodes", con)

In [6]:
counties = {'Kings':'Brooklyn', 'New York':'Manhattan', 'Queens':'Queens', 'Richmond':'Staten Island', 'Bronx':'Bronx'}

In [7]:
cuisines = []
for i in range(len(data)):
    if data[i]['parents']==['restaurants']:
        cuisines.append(data[i]['title'])

In [8]:
totals_df = pd.DataFrame(columns=['zipcode', 'county_name', 'cuisine', 'total_businesses'])

In [9]:
# conversion function:
def dict2json(dictionary):
    return json.dumps(dictionary, ensure_ascii=False)

In [10]:
for ct in counties:
    countyDf = zipcodes[(zipcodes['county_name'] == ct)]
    print(ct)
    ct_zips = countyDf[(countyDf['county_name']==ct)]['zip_code']
    
    for zp in ct_zips:
        zpstr = str(countyDf[(countyDf['zip_code']==zp)]['zip_code'].iloc[0])
        
        local = counties[ct] + ", "+zpstr
                        
        print(zp)
        print(local)
        
        for food in cuisines:
            print(food)
            offset = 0
            limit = 50
            total = 1000
            
            res_df = pd.DataFrame()
        
            while offset < total and (offset+limit)<1000:
                # define the api call
                url='https://api.yelp.com/v3/businesses/search'
                headers = {'Authorization': 'Bearer %s' % config.yelpApi}
                params = {'categories':'restaurants', 'term':food, 'location':local, 'limit':limit, 'offset':offset}
            
                # Making an initial request to the API and saving the first set of results
                req = requests.get(url, params=params, headers=headers)
                res_json = json.loads(req.text)
                res_df = res_df.append(pd.DataFrame.from_dict(res_json['businesses']))
            
                # current placement
                print('Currently at {} out of {}.'.format(params['offset'], res_json['total']))
                offset = offset+50+1
                total = res_json['total']
            
            if res_json['total'] > 0:
                totals_df = totals_df.append([{'zip_code':zp,
                                               'county_name': ct, 
                                               'cuisine':food,
                                               'total_businesses':res_json['total']}],
                                                 ignore_index=True)
                # overwrite the dict column with json-strings
                res_df['categories'] = res_df.categories.map(dict2json)
                res_df['coordinates'] = res_df.coordinates.map(dict2json)
                res_df['location'] = res_df.location.map(dict2json)
                res_df['cuisine'] = food
            
                engine = create_engine('postgresql+psycopg2:///yelp_abcs')
                con = engine.connect()
                res_df.to_sql('yelp_businesses', engine, schema = 'stag', if_exists='append')               

Kings
11221
Brooklyn, 11221
Afghan
Currently at 0 out of 82.
Currently at 51 out of 82.


/home/cusp/yg833/.conda/envs/yelpVE/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


African
Currently at 0 out of 18.
Andalusian
Currently at 0 out of 18.
Arabian
Currently at 0 out of 57.
Currently at 51 out of 57.
Argentine
Currently at 0 out of 11.
Armenian
Currently at 0 out of 44.
Asian Fusion
Currently at 0 out of 101.
Currently at 51 out of 101.
Asturian
Currently at 0 out of 10.
Australian
Currently at 0 out of 14.
Austrian
Currently at 0 out of 8.
Baguettes
Currently at 0 out of 138.
Currently at 51 out of 138.
Currently at 102 out of 138.
Bangladeshi
Currently at 0 out of 14.
Basque
Currently at 0 out of 55.
Currently at 51 out of 55.
Bavarian
Currently at 0 out of 10.
Barbeque
Currently at 0 out of 1000.
Currently at 51 out of 1000.
Currently at 102 out of 1000.
Currently at 153 out of 1000.
Currently at 204 out of 1000.
Currently at 255 out of 1000.
Currently at 306 out of 1000.
Currently at 357 out of 1000.
Currently at 408 out of 1000.
Currently at 459 out of 1000.
Currently at 510 out of 1000.
Currently at 561 out of 1000.
Currently at 612 out of 1000.


Currently at 153 out of 365.
Currently at 204 out of 365.
Currently at 255 out of 365.
Currently at 306 out of 365.
Currently at 357 out of 365.
Hot Pot
Currently at 0 out of 26.
Hungarian
Currently at 0 out of 13.
Iberian
Currently at 0 out of 37.
Indonesian
Currently at 0 out of 34.
Indian
Currently at 0 out of 89.
Currently at 51 out of 89.
International
Currently at 0 out of 177.
Currently at 51 out of 177.
Currently at 102 out of 177.
Currently at 153 out of 177.
Irish
Currently at 0 out of 87.
Currently at 51 out of 87.
Island Pub
Currently at 0 out of 510.
Currently at 51 out of 510.
Currently at 102 out of 510.
Currently at 153 out of 510.
Currently at 204 out of 510.
Currently at 255 out of 510.
Currently at 306 out of 510.
Currently at 357 out of 510.
Currently at 408 out of 510.
Currently at 459 out of 510.
Israeli
Currently at 0 out of 36.
Italian
Currently at 0 out of 457.
Currently at 51 out of 457.
Currently at 102 out of 457.
Currently at 153 out of 457.
Currently at 20

Currently at 663 out of 1800.
Currently at 714 out of 1800.
Currently at 765 out of 1800.
Currently at 816 out of 1800.
Currently at 867 out of 1800.
Currently at 918 out of 1800.
Sandwiches
Currently at 0 out of 1600.
Currently at 51 out of 1600.
Currently at 102 out of 1600.
Currently at 153 out of 1600.
Currently at 204 out of 1600.
Currently at 255 out of 1600.
Currently at 306 out of 1600.
Currently at 357 out of 1600.
Currently at 408 out of 1600.
Currently at 459 out of 1600.
Currently at 510 out of 1600.
Currently at 561 out of 1600.
Currently at 612 out of 1600.
Currently at 663 out of 1600.
Currently at 714 out of 1600.
Currently at 765 out of 1600.
Currently at 816 out of 1600.
Currently at 867 out of 1600.
Currently at 918 out of 1600.
Scandinavian
Currently at 0 out of 11.
Schnitzel
Currently at 0 out of 14.
Scottish
Currently at 0 out of 7.
Seafood
Currently at 0 out of 772.
Currently at 51 out of 772.
Currently at 102 out of 772.
Currently at 153 out of 772.
Currently at

...

Currently at 612 out of 755.
Currently at 663 out of 755.
Currently at 714 out of 755.
Venison
Currently at 0 out of 10.
Vietnamese
Currently at 0 out of 61.
Currently at 51 out of 61.
Waffles
Currently at 0 out of 204.
Currently at 51 out of 204.
Currently at 102 out of 204.
Currently at 153 out of 204.
Wok
Currently at 0 out of 108.
Currently at 51 out of 108.
Currently at 102 out of 108.
Wraps
Currently at 0 out of 845.
Currently at 51 out of 845.
Currently at 102 out of 845.
Currently at 153 out of 845.
Currently at 204 out of 845.
Currently at 255 out of 845.
Currently at 306 out of 845.
Currently at 357 out of 845.
Currently at 408 out of 845.
Currently at 459 out of 845.
Currently at 510 out of 845.
Currently at 561 out of 845.
Currently at 612 out of 845.
Currently at 663 out of 845.
Currently at 714 out of 845.
Currently at 765 out of 845.
Currently at 816 out of 845.
Yugoslav
Currently at 0 out of 1.
11202
Brooklyn, 11202
Afghan
Currently at 0 out of 11.
African
Currently at 

Currently at 51 out of 102.
Curry Sausage
Currently at 0 out of 487.
Currently at 51 out of 487.
Currently at 102 out of 487.
Currently at 153 out of 487.
Currently at 204 out of 487.
Currently at 255 out of 487.
Currently at 306 out of 487.
Currently at 357 out of 487.
Currently at 408 out of 487.
Currently at 459 out of 487.
Cypriot
Currently at 0 out of 11.
Czech
Currently at 0 out of 15.
Czech/Slovakian
Currently at 0 out of 3.
Danish
Currently at 0 out of 302.
Currently at 51 out of 302.
Currently at 102 out of 302.
Currently at 153 out of 302.
Currently at 204 out of 302.
Currently at 255 out of 302.
Delis
Currently at 0 out of 684.
Currently at 51 out of 684.
Currently at 102 out of 684.
Currently at 153 out of 684.
Currently at 204 out of 684.
Currently at 255 out of 684.
Currently at 306 out of 684.
Currently at 357 out of 684.
Currently at 408 out of 684.
Currently at 459 out of 684.
Currently at 510 out of 684.
Currently at 561 out of 684.
Currently at 612 out of 684.
Curren

Currently at 51 out of 290.
Currently at 102 out of 290.
Currently at 153 out of 290.
Currently at 204 out of 290.
Currently at 255 out of 290.
Kopitiam
Currently at 0 out of 6.
Korean
Currently at 0 out of 318.
Currently at 51 out of 318.
Currently at 102 out of 318.
Currently at 153 out of 318.
Currently at 204 out of 318.
Currently at 255 out of 318.
Currently at 306 out of 318.
Kosher
Currently at 0 out of 388.
Currently at 51 out of 388.
Currently at 102 out of 388.
Currently at 153 out of 388.
Currently at 204 out of 388.
Currently at 255 out of 388.
Currently at 306 out of 388.
Currently at 357 out of 388.
Kurdish
Currently at 0 out of 6.
Laos
Currently at 0 out of 20.
Laotian
Currently at 0 out of 11.
Latin American
Currently at 0 out of 162.
Currently at 51 out of 162.
Currently at 102 out of 162.
Currently at 153 out of 162.
Lyonnais
Currently at 0 out of 8.
Malaysian
Currently at 0 out of 96.
Currently at 51 out of 96.
Meatballs
Currently at 0 out of 1500.
Currently at 51 ou

Currently at 102 out of 3400.
Currently at 153 out of 3400.
Currently at 204 out of 3400.
Currently at 255 out of 3400.
Currently at 306 out of 3400.
Currently at 357 out of 3400.
Currently at 408 out of 3400.
Currently at 459 out of 3400.
Currently at 510 out of 3400.
Currently at 561 out of 3400.
Currently at 612 out of 3400.
Currently at 663 out of 3400.
Currently at 714 out of 3400.
Currently at 765 out of 3400.
Currently at 816 out of 3400.
Currently at 867 out of 3400.
Currently at 918 out of 3400.
Romanian
Currently at 0 out of 22.
Rotisserie Chicken
Currently at 0 out of 163.
Currently at 51 out of 163.
Currently at 102 out of 163.
Currently at 153 out of 163.
Russian
Currently at 0 out of 38.
Salad
Currently at 0 out of 3800.
Currently at 51 out of 3800.
Currently at 102 out of 3800.
Currently at 153 out of 3800.
Currently at 204 out of 3800.
Currently at 255 out of 3800.
Currently at 306 out of 3800.
Currently at 357 out of 3800.
Currently at 408 out of 3800.
Currently at 459

KeyError: 'businesses'

In [11]:
res_json

{'error': {'code': 'INTERNAL_ERROR',
  'description': 'Something went wrong internally, please try again later.'}}